In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import sqlite3

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Set up the browser
u_id = 100
driver = webdriver.Chrome()
driver.get("https://www.daraz.pk/smartphones/")

count = 0
username = []
rating = []
uniqueid = []
name = []
price = []
questions = []
brand = []
reviews = []
specifications = []
total_rating = []
Links = []

# Iterate through elements
for i in range(2, 7):
    try:
        if i != 2:
            
            element = driver.find_element(By.XPATH, f'//*[@id="root"]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[{i}]')
            element.click()
        
        for j in range(1, 39):

            driver.execute_script(f"window.scrollTo(0, {j*50});")
            u_id += 1

            div = driver.find_element(By.XPATH, f"/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[2]/div[{j}]/div/div/div[1]/div/a/img")
            div.click()

            # id
            try:
                uniqueid.append(u_id)
            except:
                uniqueid.append(np.nan)


            try:
                # Get the link corresponding to the unique identifier
                link_for_id = driver.current_url
                Links.append(link_for_id)
            except:
                Links.append(np.nan)


            # questions
            try:
                questions.append(driver.find_element(By.XPATH, '//*[@id="module_product_review_star_1"]/div/a[2]').text)
            except:
                questions.append(np.nan)
            # name
            try:
                name.append(driver.find_element(By.XPATH, '//*[@id="module_product_title_1"]/div/div/span').text)
            except:
                name.append(np.nan)
            # price
            try:
                price.append(driver.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/span').text)
            except:
                price.append(np.nan)
            # brand
            try:
                brand.append(driver.find_element(By.XPATH, '//*[@id="module_product_brand_1"]/div/a[1]').text)
            except:
                brand.append(np.nan)
            # rating
            try:
                driver.execute_script("window.scrollTo(0, 800);")
                time.sleep(2)
                rating.append(driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]/span[1]').text)
            except:
                rating.append(np.nan)
             # Total rating
            try:
                total_rating.append(driver.find_element(By.XPATH, '//*[@id="module_product_review_star_1"]/div/a[1]').text)
            except:
                total_rating.append(np.nan)

            # REVIEWS
            driver.execute_script("window.scrollTo(0, 1100);")
            time.sleep(2)
            revs = driver.find_elements(By.CLASS_NAME, "review-content-sl")
            u_name = driver.find_elements(By.CLASS_NAME, "user")
            temp1 = [rev.text for rev in revs]
            temp3 = [aa.text for aa in u_name]
            reviews.append(temp1)
            username.append(temp3)



            # SPECIFICATIONS
            specs = driver.find_elements(By.CLASS_NAME, "html-content.pdp-product-highlights")
            temp2 = [spec.text for spec in specs]
            specifications.append(temp2)

            driver.back()
            count += 1

    except Exception as e:
        print(f"Error: {e}")

# Close the browser
driver.quit()


In [15]:
# Create a dictionary with your lists
data2 = {
    'Id':uniqueid,
    'Reviews': reviews,
    'UserName': username
}
data1 = {
    'Id':uniqueid,
    'Rating':rating,
    'Total Rating':total_rating,
    'Name': name,
    'Price': price,
    'Brand': brand,
    'Specifications': specifications,
    'Questions': questions,
    'Links':Links

}


# Create a DataFrame from the dictionary
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

###################################################### IRRELEVENT PRODUCTS  ##########################################################

Filter COVERS / CASES / CHARGERS 
keywords_to_exclude = ['case', 'charger', 'cover']
# Remove rows containing keywords in 'Name' column
df1 = df1[~df1['Name'].str.contains('|'.join(keywords_to_exclude), case=False)]
# Remove corresponding rows from df2 based on 'Id'
df2 = df2[df2['Id'].isin(df1['Id'])]


###################################################### DATA CLEANING  ##########################################################

df1 =  df1.drop_duplicates()
df2 =  df2.drop_duplicates()

df1['Price'] = df1['Price'].str.replace(',', '')
df1['Price'] = df1['Price'].str.extract('(\d+)')
df1['Total Rating'] = df1['Total Rating'].str.extract('(\d+)')
df1['Questions'] = df1['Questions'].str.extract('(\d+)')


df1['Rating'] = pd.to_numeric(df1['Rating'], errors='coerce')
df1['Total Rating'] = pd.to_numeric(df1['Total Rating'], errors='coerce')


# Get the top 5 rows based on 'Rating'
best = df1.nlargest(5, ['Rating', 'Total Rating'])


###################################################  WRITE TO DATABASE ##########################################################
#Connect to SQLite database
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# Create df1 table
df1.to_sql('df1', conn, index=False, if_exists='replace')

# Create df2 table
df2.to_sql('df2', conn, index=False, if_exists='replace')

# Commit and close the connection
conn.commit()
conn.close()

#Write to drive
df1.to_csv('product.csv')
df2.to_csv('reviews.csv')
best.to_csv('best.csv')


# Display the DataFrame
print(df1)
print(df2)

ProgrammingError: Error binding parameter 7: type 'list' is not supported